In [54]:
import random
from typing import TypedDict
import statistics
import numpy as np

In [55]:
PET_LEVEL_TO_DROPRATE_MULTIPLIER = {
    0:1.0,
    1:1.0,
    2:1.0,
    3:1.0,
    4:1.33,
    5:1.67,
    6:2.0,
    7:2.33,
    8:2.67,
    9:3.0,
    10:3.33,
    11:3.67,
    12:4.0,
}

class PetDropRatesDict(TypedDict):
    item: str
    dropRate: int


class Pet:
    def __init__(self, name:str, drops_dict: PetDropRatesDict, cooldown: int):
        self.name = name
        self.drops = drops_dict
        self.cooldown = cooldown


class EndSimDropConditions(TypedDict):
    item: str
    required_total: int



In [56]:
classic_dragon_pet_drops: PetDropRatesDict = {
    "Hoard":100,
    "Site Card":150,
    "Event Card":500
}
classic_dragon_pet = Pet("Classic Dragon", classic_dragon_pet_drops, cooldown=23)

black_dragon_pet_drops: PetDropRatesDict = {
    "Hoard":100,
    "Site Card":150,
    "Event Card":450
}
black_dragon_pet = Pet("Black Dragon", black_dragon_pet_drops, cooldown=23)

ada_dwarf_drops: PetDropRatesDict = {
    "Adamantium Ore":200,
}
ada_dwarf_pet = Pet("Adamantium Dwarf", ada_dwarf_drops, cooldown=24)

quartz_dwarf_drops: PetDropRatesDict = {
    "Quartz Ore":65,
    "Dwarven Gem":300,
}
quartz_dwarf_pet = Pet("Quartz Dwarf", quartz_dwarf_drops, cooldown=24)



In [57]:
def run_simulation(sim_days: int, pet:Pet, pet_level:int):
    simulation_steps = sim_days*24
    pet_level_drop_rate_modifier = PET_LEVEL_TO_DROPRATE_MULTIPLIER[pet_level]
    
    for key in pet.drops.keys():
        chance_to_drop = pet_level_drop_rate_modifier/pet.drops.get(key)
        #print(f'Chance to drop {key} is {pet_level_drop_rate_modifier}/{pet.drops.get(key)} = {pet_level_drop_rate_modifier/pet.drops.get(key)} ')

    drop_results = dict() # stores list of total days since last drop per each item
    steps_since_last_drop = dict() # stores temporary days since last drop count for each item 
    for drop_item in pet.drops.keys():
        drop_results[drop_item] = list()
        steps_since_last_drop[drop_item] = 0

    last_drop_step = 0
    for sim_step in range(simulation_steps):
        if((sim_step - last_drop_step) < pet.cooldown):
            #print(f'Sim Step is  {sim_step}, last item drop is {last_drop_step} continuing!')
            for drop_item in pet.drops.keys(): #increase steps since last drop for each drop item
                steps_since_last_drop[drop_item] += 1
            continue #move to next sim step

        for drop_item in pet.drops.keys():
            chance_to_drop = pet_level_drop_rate_modifier/pet.drops.get(drop_item)
            item_dropped = random.random() < chance_to_drop
            if(item_dropped):
                #print(f'Item {drop_item} dropped after {steps_since_last_drop[drop_item]} steps')
                drop_results[drop_item].append(steps_since_last_drop[drop_item]+1) # add one more step to account for this step
                steps_since_last_drop[drop_item] = 0 # reset steps since the last time this item dropped
                last_drop_step = sim_step # set the global last time any item dropped
            else: # no item drop, add step to its count since last drop
                steps_since_last_drop[drop_item] += 1
    
    return drop_results
            

In [58]:
def run_n_sim(num_sims:int, years_per_sim:int, pet:Pet, pet_level:int):
    all_results = dict()
    #combine drop time results from all simulations
    for n in range(num_sims):
        for drop_item in pet.drops.keys():
            all_results[drop_item] = list()
        
        drop_results = run_simulation(years_per_sim*365, pet, pet_level)
        for drop_item in pet.drops.keys():
            all_results[drop_item].extend(drop_results.get(drop_item))
    return all_results

In [ ]:
def print_n_sims_result_sim_info(pet:Pet, pet_level, all_results):
    num_sims = len(all_results['__TOTAL_SIM_TIME']) # get num sims from length of array to double-check
    drop_string = f'SIM PET: {pet.name} || SIM LEVEL {pet_level} || NUM SIMS: {num_sims}:\n ' + \
            f'Avg: {round(avg_drop_days,2)} StdDev: {round(stddev_drop_days,2)} ' + \
            f'Median: {round(median_drop_days,2)} 95%ile: {round(percentile95_drop_days,2)} '+ \
            f'Min: {round(min_drop_days,2)} Max: {round(max_drop_days,2)} '
    print(drop_string)



def print_n_sims_result(pet:Pet, pet_level, all_results):
    # simulation output is in steps, convert steps to days while printing stats 
    for drop_item in pet.drops.keys():
        avg_drop_days = np.mean(all_results.get(drop_item))/24
        stddev_drop_days = np.std(all_results.get(drop_item))/24
        min_drop_days = np.min(all_results.get(drop_item))/24
        max_drop_days = np.max(all_results.get(drop_item))/24
        median_drop_days = np.median(all_results.get(drop_item))/24
        percentile95_drop_days = np.percentile(all_results.get(drop_item),95)/24

        drop_string = f'TIME BETWEEN DROPS [DAYS] {pet.name} @ LEVEL {pet_level} DROP: {drop_item}:\n ' + \
                    f'Avg: {round(avg_drop_days,2)} StdDev: {round(stddev_drop_days,2)} ' + \
                    f'Median: {round(median_drop_days,2)} 95%ile: {round(percentile95_drop_days,2)} '+ \
                    f'Min: {round(min_drop_days,2)} Max: {round(max_drop_days,2)} '
        print(drop_string)

def print_n_sims_results_total_drops(pet:Pet, pet_level, all_results):
    # simulation output is in steps, convert steps to days while printing stats 
    for drop_item in pet.drops.keys():
        avg_total_drops = np.mean(all_results['__ITEM_TOTAL_DROPS'][drop_item])
        stddev_total_drops = np.std(all_results['__ITEM_TOTAL_DROPS'][drop_item])
        min_total_drops = np.min(all_results['__ITEM_TOTAL_DROPS'][drop_item])
        max_total_drops = np.max(all_results['__ITEM_TOTAL_DROPS'][drop_item])
        median_total_drops = np.median(all_results['__ITEM_TOTAL_DROPS'][drop_item])
        percentile95_total_drops = np.percentile(all_results['__ITEM_TOTAL_DROPS'][drop_item],95)

        drop_string = f'TOTAL DROPS [COUNT] {pet.name} @ LEVEL {pet_level} DROP: {drop_item}:\n ' + \
                    f'Avg: {round(avg_total_drops,2)} StdDev: {round(stddev_total_drops,2)} ' + \
                    f'Median: {round(median_total_drops,2)} 95%ile: {round(percentile95_total_drops,2)} '+ \
                    f'Min: {round(min_total_drops,2)} Max: {round(max_total_drops,2)} '
        print(drop_string)

def print_n_sims_results_sim_time(pet:Pet, pet_level, all_results):
    num_sims = len(all_results['__TOTAL_SIM_TIME']) # get num sims from length of array to double-check

    # simulation output is in steps, convert steps to days while printing stats 
    avg_sim_time = np.mean(all_results['__TOTAL_SIM_TIME'])/24
    stddev_sim_time = np.std(all_results['__TOTAL_SIM_TIME'])/24
    min_sim_time = np.min(all_results['__TOTAL_SIM_TIME'])/24
    max_sim_time = np.max(all_results['__TOTAL_SIM_TIME'])/24
    median_sim_time = np.median(all_results['__TOTAL_SIM_TIME'])/24
    percentile95_sim_time = np.percentile(all_results['__TOTAL_SIM_TIME'],95)/24

    drop_string = f'SIM PET: {pet.name} || SIM LEVEL {pet_level} || NUM SIMS: {num_sims}:\n ' + \
                f'TOTAL SIM TIME [DAYS] STATS:\n ' + \
                f'Avg: {round(avg_sim_time,2)} StdDev: {round(stddev_sim_time,2)} ' + \
                f'Median: {round(median_sim_time,2)} 95%ile: {round(percentile95_sim_time,2)} '+ \
                f'Min: {round(min_sim_time,2)} Max: {round(max_sim_time,2)} '
    print(drop_string)

In [ ]:
NUM_SIMS = 10000
SIM_YEARS = 30
PET = classic_dragon_pet

for level in range(12,13):
    level_results = run_n_sim(NUM_SIMS, SIM_YEARS, PET, level)
    print_n_sims_result(PET, level, level_results)
    print()

In [ ]:
NUM_SIMS = 10000
SIM_YEARS = 30
SIM_PET_DROPS = black_dragon_pet

for level in range(12,13):
    level_results = run_n_sim(NUM_SIMS, SIM_YEARS, SIM_PET_DROPS, level)
    print_n_sims_result(SIM_PET_DROPS, level, level_results)
    print()

Black Dragon @ Level  12 - stats [days] for Hoard:
 Avg: 2.68 StdDev: 1.93 Median: 2.12 95%ile: 6.5 Min: 0.96 Max: 20.0 
Black Dragon @ Level  12 - stats [days] for Site Card:
 Avg: 4.12 StdDev: 3.4 Median: 3.08 95%ile: 10.96 Min: 0.96 Max: 29.25 
Black Dragon @ Level  12 - stats [days] for Event Card:
 Avg: 12.4 StdDev: 11.62 Median: 8.83 95%ile: 35.58 Min: 0.96 Max: 77.42 



In [72]:
def run_simulation_until_drop_condition(max_sim_days: int, pet:Pet, pet_level:int, end_sim_drop_conditions: EndSimDropConditions):
    simulation_steps = max_sim_days*24
    pet_level_drop_rate_modifier = PET_LEVEL_TO_DROPRATE_MULTIPLIER[pet_level]
    
    for key in pet.drops.keys():
        chance_to_drop = pet_level_drop_rate_modifier/pet.drops.get(key)
        #print(f'Chance to drop {key} is {pet_level_drop_rate_modifier}/{pet.drops.get(key)} = {pet_level_drop_rate_modifier/pet.drops.get(key)} ')

    drop_results = dict() # stores list of total days since last drop per each item
    steps_since_last_drop = dict() # stores temporary days since last drop count for each item 
    for drop_item in pet.drops.keys():
        drop_results[drop_item] = list()
        steps_since_last_drop[drop_item] = 0

    last_drop_step = 0
    for sim_step in range(simulation_steps):
        if((sim_step - last_drop_step) < pet.cooldown):
            #print(f'Sim Step is  {sim_step}, last item drop is {last_drop_step} continuing!')
            for drop_item in pet.drops.keys(): #increase steps since last drop for each drop item
                steps_since_last_drop[drop_item] += 1
            continue #move to next sim step

        for drop_item in pet.drops.keys():
            chance_to_drop = pet_level_drop_rate_modifier/pet.drops.get(drop_item)
            item_dropped = random.random() < chance_to_drop
            if(item_dropped):
                #print(f'Item {drop_item} dropped after {steps_since_last_drop[drop_item]} steps')
                drop_results[drop_item].append(steps_since_last_drop[drop_item]+1) # add one more step to account for this step
                steps_since_last_drop[drop_item] = 0 # reset steps since the last time this item dropped
                last_drop_step = sim_step # set the global last time any item dropped
            else: # no item drop, add step to its count since last drop
                steps_since_last_drop[drop_item] += 1
        
        end_sim_early = True
        for end_sim_drop_item in end_sim_drop_conditions.keys():
            try:
                current_list_of_item_drops = drop_results[end_sim_drop_item]
            except:
                raise Exception(f"Could not access {end_sim_drop_item} in list of drops")
            
            if(len(current_list_of_item_drops) < end_sim_drop_conditions[end_sim_drop_item]):
                end_sim_early = False
        
        if end_sim_early: # all conditons for ending early met, end simulation
            break
    
    return sim_step, drop_results
            

In [73]:
def run_n_sim_with_stop(num_sims:int, years_per_sim:int, pet:Pet, pet_level:int, end_sim_drop_conditions: EndSimDropConditions):
    all_results = dict()
    all_results['__TOTAL_SIM_TIME'] = list() # stores total sim time per simulation
    all_results['__ITEM_TOTAL_DROPS'] = dict() # stores per-item total drops for each sim

    for drop_item in pet.drops.keys():
        all_results[drop_item] = list() # days per drop for item across all sims
        all_results['__ITEM_TOTAL_DROPS'][drop_item] = list() # stores per-item total drops for each sim

    #combine drop time results from all simulations
    for _ in range(num_sims):
        sim_step, drop_results = run_simulation_until_drop_condition(years_per_sim*365, pet, pet_level, end_sim_drop_conditions)
        for drop_item in pet.drops.keys():
            all_results[drop_item].extend(drop_results.get(drop_item))
            all_results['__ITEM_TOTAL_DROPS'][drop_item].append(len(drop_results.get(drop_item))) # append number of total drops for this item
        all_results['__TOTAL_SIM_TIME'].append(sim_step) #append total sim time for this sim
        
    return all_results

In [79]:
NUM_SIMS = 30000
SIM_YEARS = 30
PET = classic_dragon_pet
LEVEL = 0

STOP_EARLY_CONDITIONS : EndSimDropConditions = {
    "Hoard":19,
}

level_results = run_n_sim_with_stop(NUM_SIMS, SIM_YEARS, PET, LEVEL, STOP_EARLY_CONDITIONS)
print_n_sims_results_sim_time(PET, LEVEL, level_results)
print()
print_n_sims_results_total_drops(PET, LEVEL, level_results)
print()
print_n_sims_result(PET, LEVEL, level_results)
print()


SIM PET: Classic Dragon || SIM LEVEL 0 || NUM SIMS: 30000:
 Avg: 111.61 StdDev: 21.82 Median: 110.0 95%ile: 150.04 Min: 46.46 Max: 254.17 

TOTAL DROPS [COUNT] Classic Dragon @ LEVEL 0 DROP: Hoard:
 Avg: 19.0 StdDev: 0.0 Median: 19.0 95%ile: 19.0 Min: 19 Max: 19 
TOTAL DROPS [COUNT] Classic Dragon @ LEVEL 0 DROP: Site Card:
 Avg: 12.69 StdDev: 4.57 Median: 12.0 95%ile: 21.0 Min: 0 Max: 36 
TOTAL DROPS [COUNT] Classic Dragon @ LEVEL 0 DROP: Event Card:
 Avg: 3.8 StdDev: 2.12 Median: 4.0 95%ile: 8.0 Min: 0 Max: 18 

TIME BETWEEN DROPS [DAYS] Classic Dragon @ LEVEL 0 DROP: Hoard:
 Avg: 5.88 StdDev: 5.0 Median: 4.33 95%ile: 15.88 Min: 0.96 Max: 65.71 
TIME BETWEEN DROPS [DAYS] Classic Dragon @ LEVEL 0 DROP: Site Card:
 Avg: 8.11 StdDev: 7.2 Median: 5.92 95%ile: 22.58 Min: 0.96 Max: 95.88 
TIME BETWEEN DROPS [DAYS] Classic Dragon @ LEVEL 0 DROP: Event Card:
 Avg: 21.93 StdDev: 19.4 Median: 16.17 95%ile: 61.96 Min: 0.96 Max: 169.0 



In [80]:
NUM_SIMS = 30000
SIM_YEARS = 30
PET = quartz_dwarf_pet
LEVEL = 0

STOP_EARLY_CONDITIONS : EndSimDropConditions = {
    "Dwarven Gem":5,
}

level_results = run_n_sim_with_stop(NUM_SIMS, SIM_YEARS, PET, LEVEL, STOP_EARLY_CONDITIONS)
print_n_sims_results_sim_time(PET, LEVEL, level_results)
print()
print_n_sims_results_total_drops(PET, LEVEL, level_results)
print()
print_n_sims_result(PET, LEVEL, level_results)
print()


SIM PET: Quartz Dwarf || SIM LEVEL 0 || NUM SIMS: 30000:
 Avg: 89.45 StdDev: 38.12 Median: 83.62 95%ile: 160.79 Min: 9.29 Max: 336.62 

TOTAL DROPS [COUNT] Quartz Dwarf @ LEVEL 0 DROP: Quartz Ore:
 Avg: 23.15 StdDev: 11.35 Median: 22.0 95%ile: 44.0 Min: 0 Max: 100 
TOTAL DROPS [COUNT] Quartz Dwarf @ LEVEL 0 DROP: Dwarven Gem:
 Avg: 5.0 StdDev: 0.0 Median: 5.0 95%ile: 5.0 Min: 5 Max: 5 

TIME BETWEEN DROPS [DAYS] Quartz Dwarf @ LEVEL 0 DROP: Quartz Ore:
 Avg: 3.7 StdDev: 2.74 Median: 2.88 95%ile: 9.17 Min: 1.0 Max: 36.96 
TIME BETWEEN DROPS [DAYS] Quartz Dwarf @ LEVEL 0 DROP: Dwarven Gem:
 Avg: 17.9 StdDev: 17.08 Median: 12.67 95%ile: 52.04 Min: 1.0 Max: 195.33 

